In [2]:
import os
import pickle
import pandas as pd
from pathlib import Path

In [4]:
# 피클 파일
path = "C:/Users/user/Desktop/박준영/공부/nlp_study/simcse/data/pickle_180_days_compare_kospi/yes_data/"
data_list = os.listdir(path)
len(data_list)

data_dict = {}


for data_name in data_list:
    with open(path + data_name, 'rb') as f:
        data_dict[data_name] = pickle.load(f)

In [12]:
data_dict['20171215.pkl'].loc[:, 0:] = data_dict['20171215.pkl'].loc[:, 0:].astype(float)

C:\Users\user\AppData\Local\Temp\ipykernel_748\723239555.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_dict['20171215.pkl'].loc[:, 0:] = data_dict['20171215.pkl'].loc[:, 0:].astype(float)


In [26]:
imp = data_dict['20171215.pkl'].loc[:, 0:]

In [27]:
imp1 = imp.transpose().corr()

In [30]:
imp1

주가번호,393929,394109,393906,393966,394151,393918,394044,394083,395295,393952,...,393956,394006,393857,393928,394142,394051,393864,393984,393872,393933
주가번호,,,,,,,,,,,,,,,,,,,,,
393929,1.000000,-0.008298,-0.417225,-0.417225,-0.063009,-0.001407,-0.070542,0.260125,-0.080035,0.069122,...,0.271351,0.271351,0.271351,0.333319,0.060844,-0.012387,0.304240,-0.115197,0.763380,0.568635
394109,-0.008298,1.000000,0.238349,0.238349,0.199079,-0.224174,0.003953,0.253057,-0.002554,-0.037892,...,0.320023,0.320023,0.320023,-0.172853,-0.404390,0.109599,-0.229602,0.400705,-0.234779,-0.246623
393906,-0.417225,0.238349,1.000000,1.000000,0.669440,-0.456160,0.534070,-0.554866,0.221077,-0.406531,...,0.434106,0.434106,0.434106,-0.576944,-0.526351,0.445831,-0.456878,0.127329,-0.602129,-0.643461
393966,-0.417225,0.238349,1.000000,1.000000,0.669440,-0.456160,0.534070,-0.554866,0.221077,-0.406531,...,0.434106,0.434106,0.434106,-0.576944,-0.526351,0.445831,-0.456878,0.127329,-0.602129,-0.643461
394151,-0.063009,0.199079,0.669440,0.669440,1.000000,-0.621716,0.617574,-0.304267,0.486657,-0.534185,...,0.612491,0.612491,0.612491,-0.356819,-0.514938,0.474927,0.177898,0.112446,-0.196414,-0.335862
393918,-0.001407,-0.224174,-0.456160,-0.456160,-0.621716,1.000000,-0.819198,0.378450,-0.627587,0.641490,...,-0.553759,-0.553759,-0.553759,-0.043393,0.137878,-0.709577,0.016258,-0.455729,0.380723,-0.061208
394044,-0.070542,0.003953,0.534070,0.534070,0.617574,-0.819198,1.000000,-0.440428,0.623336,-0.689812,...,0.457693,0.457693,0.457693,-0.021130,-0.116223,0.728668,-0.128443,0.303188,-0.438538,0.058214
394083,0.260125,0.253057,-0.554866,-0.554866,-0.304267,0.378450,-0.440428,1.000000,-0.064474,0.472180,...,-0.362995,-0.362995,-0.362995,0.112132,0.135661,-0.468350,0.308536,-0.095140,0.327041,0.186889
395295,-0.080035,-0.002554,0.221077,0.221077,0.486657,-0.627587,0.623336,-0.064474,1.000000,-0.319789,...,0.099547,0.099547,0.099547,-0.091606,0.222714,0.429687,0.351908,0.031281,-0.321890,-0.048927


In [31]:
# 각 행별로 가장 높은 및 낮은 상관관계를 가진 다른 행을 찾고 그 데이터를 추출
for row in imp.index:
    # 해당 행에 대한 상관관계 값
    current_row_correlations = imp1.loc[row].drop(row)
    
    # 최대 및 최소 상관관계 값을 가진 행 이름 찾기
    min_corr_row = current_row_correlations.idxmin()
    max_corr_row = current_row_correlations.idxmax()
    
    # 데이터 추출 및 출력
    print(f"{row}와 가장 낮은 상관관계를 갖는 행: {min_corr_row}")
    # print(imp.loc[min_corr_row], "\n")
    
    print(f"{row}와 가장 높은 상관관계를 갖는 행: {max_corr_row}")
    # print(imp.loc[max_corr_row], "\n")
    print("------")

393929와 가장 낮은 상관관계를 갖는 행: 393847
393929와 가장 높은 상관관계를 갖는 행: 393872
------
394109와 가장 낮은 상관관계를 갖는 행: 394142
394109와 가장 높은 상관관계를 갖는 행: 393847
------
393906와 가장 낮은 상관관계를 갖는 행: 393950
393906와 가장 높은 상관관계를 갖는 행: 393966
------
393966와 가장 낮은 상관관계를 갖는 행: 393950
393966와 가장 높은 상관관계를 갖는 행: 393906
------
394151와 가장 낮은 상관관계를 갖는 행: 393918
394151와 가장 높은 상관관계를 갖는 행: 393913
------
393918와 가장 낮은 상관관계를 갖는 행: 394044
393918와 가장 높은 상관관계를 갖는 행: 395294
------
394044와 가장 낮은 상관관계를 갖는 행: 395294
394044와 가장 높은 상관관계를 갖는 행: 393913
------
394083와 가장 낮은 상관관계를 갖는 행: 393913
394083와 가장 높은 상관관계를 갖는 행: 395292
------
395295와 가장 낮은 상관관계를 갖는 행: 393918
395295와 가장 높은 상관관계를 갖는 행: 395293
------
393952와 가장 낮은 상관관계를 갖는 행: 393913
393952와 가장 높은 상관관계를 갖는 행: 395294
------
393943와 가장 낮은 상관관계를 갖는 행: 393906
393943와 가장 높은 상관관계를 갖는 행: 393879
------
394042와 가장 낮은 상관관계를 갖는 행: 393913
394042와 가장 높은 상관관계를 갖는 행: 395294
------
395294와 가장 낮은 상관관계를 갖는 행: 394044
395294와 가장 높은 상관관계를 갖는 행: 394042
------
394210와 가장 낮은 상관관계를 갖는 행: 393950
394210와 가장 높은 상관관계

In [20]:
min_corr_row

394071

In [13]:
data_dict['20171215.pkl'].loc[:, 0:].transpose().corr()

주가번호,393929,394109,393906,393966,394151,393918,394044,394083,395295,393952,...,393956,394006,393857,393928,394142,394051,393864,393984,393872,393933
주가번호,,,,,,,,,,,,,,,,,,,,,
393929,1.000000,-0.008298,-0.417225,-0.417225,-0.063009,-0.001407,-0.070542,0.260125,-0.080035,0.069122,...,0.271351,0.271351,0.271351,0.333319,0.060844,-0.012387,0.304240,-0.115197,0.763380,0.568635
394109,-0.008298,1.000000,0.238349,0.238349,0.199079,-0.224174,0.003953,0.253057,-0.002554,-0.037892,...,0.320023,0.320023,0.320023,-0.172853,-0.404390,0.109599,-0.229602,0.400705,-0.234779,-0.246623
393906,-0.417225,0.238349,1.000000,1.000000,0.669440,-0.456160,0.534070,-0.554866,0.221077,-0.406531,...,0.434106,0.434106,0.434106,-0.576944,-0.526351,0.445831,-0.456878,0.127329,-0.602129,-0.643461
393966,-0.417225,0.238349,1.000000,1.000000,0.669440,-0.456160,0.534070,-0.554866,0.221077,-0.406531,...,0.434106,0.434106,0.434106,-0.576944,-0.526351,0.445831,-0.456878,0.127329,-0.602129,-0.643461
394151,-0.063009,0.199079,0.669440,0.669440,1.000000,-0.621716,0.617574,-0.304267,0.486657,-0.534185,...,0.612491,0.612491,0.612491,-0.356819,-0.514938,0.474927,0.177898,0.112446,-0.196414,-0.335862
393918,-0.001407,-0.224174,-0.456160,-0.456160,-0.621716,1.000000,-0.819198,0.378450,-0.627587,0.641490,...,-0.553759,-0.553759,-0.553759,-0.043393,0.137878,-0.709577,0.016258,-0.455729,0.380723,-0.061208
394044,-0.070542,0.003953,0.534070,0.534070,0.617574,-0.819198,1.000000,-0.440428,0.623336,-0.689812,...,0.457693,0.457693,0.457693,-0.021130,-0.116223,0.728668,-0.128443,0.303188,-0.438538,0.058214
394083,0.260125,0.253057,-0.554866,-0.554866,-0.304267,0.378450,-0.440428,1.000000,-0.064474,0.472180,...,-0.362995,-0.362995,-0.362995,0.112132,0.135661,-0.468350,0.308536,-0.095140,0.327041,0.186889
395295,-0.080035,-0.002554,0.221077,0.221077,0.486657,-0.627587,0.623336,-0.064474,1.000000,-0.319789,...,0.099547,0.099547,0.099547,-0.091606,0.222714,0.429687,0.351908,0.031281,-0.321890,-0.048927


In [9]:
imp.transpose().corr()

주가번호,393929,394109,393906,393966,394151,393918,394044,394083,395295,393952,...,393956,394006,393857,393928,394142,394051,393864,393984,393872,393933
주가번호,,,,,,,,,,,,,,,,,,,,,
393929,1.000000,-0.008298,-0.417225,-0.417225,-0.063009,-0.001407,-0.070542,0.260125,-0.080035,0.069122,...,0.271351,0.271351,0.271351,0.333319,0.060844,-0.012387,0.304240,-0.115197,0.763380,0.568635
394109,-0.008298,1.000000,0.238349,0.238349,0.199079,-0.224174,0.003953,0.253057,-0.002554,-0.037892,...,0.320023,0.320023,0.320023,-0.172853,-0.404390,0.109599,-0.229602,0.400705,-0.234779,-0.246623
393906,-0.417225,0.238349,1.000000,1.000000,0.669440,-0.456160,0.534070,-0.554866,0.221077,-0.406531,...,0.434106,0.434106,0.434106,-0.576944,-0.526351,0.445831,-0.456878,0.127329,-0.602129,-0.643461
393966,-0.417225,0.238349,1.000000,1.000000,0.669440,-0.456160,0.534070,-0.554866,0.221077,-0.406531,...,0.434106,0.434106,0.434106,-0.576944,-0.526351,0.445831,-0.456878,0.127329,-0.602129,-0.643461
394151,-0.063009,0.199079,0.669440,0.669440,1.000000,-0.621716,0.617574,-0.304267,0.486657,-0.534185,...,0.612491,0.612491,0.612491,-0.356819,-0.514938,0.474927,0.177898,0.112446,-0.196414,-0.335862
393918,-0.001407,-0.224174,-0.456160,-0.456160,-0.621716,1.000000,-0.819198,0.378450,-0.627587,0.641490,...,-0.553759,-0.553759,-0.553759,-0.043393,0.137878,-0.709577,0.016258,-0.455729,0.380723,-0.061208
394044,-0.070542,0.003953,0.534070,0.534070,0.617574,-0.819198,1.000000,-0.440428,0.623336,-0.689812,...,0.457693,0.457693,0.457693,-0.021130,-0.116223,0.728668,-0.128443,0.303188,-0.438538,0.058214
394083,0.260125,0.253057,-0.554866,-0.554866,-0.304267,0.378450,-0.440428,1.000000,-0.064474,0.472180,...,-0.362995,-0.362995,-0.362995,0.112132,0.135661,-0.468350,0.308536,-0.095140,0.327041,0.186889
395295,-0.080035,-0.002554,0.221077,0.221077,0.486657,-0.627587,0.623336,-0.064474,1.000000,-0.319789,...,0.099547,0.099547,0.099547,-0.091606,0.222714,0.429687,0.351908,0.031281,-0.321890,-0.048927


In [10]:
for i in range(imp.transpose().corr().shape[0]):
    for j in range(imp.transpose().corr().shape[1]):
        if i >= j:
            imp.transpose().corr().iloc[i, j] = float('nan')

# 가장 낮은 및 높은 상관관계 찾기
min_corr_value = imp.transpose().corr().min().min()
max_corr_value = imp.transpose().corr().max().max()

min_corr_pairs = correlation_matrix_rows[correlation_matrix_rows == min_corr_value].stack().index.tolist()
max_corr_pairs = correlation_matrix_rows[correlation_matrix_rows == max_corr_value].stack().index.tolist()

print("가장 낮은 상관관계:", min_corr_value, "행 쌍:", min_corr_pairs)
print("가장 높은 상관관계:", max_corr_value, "행 쌍:", max_corr_pairs)

NameError: name 'correlation_matrix_rows' is not defined